In [53]:
from datetime import datetime
import pandas as pd

# HELPER FUNCTIONS

In [54]:
def quarter_dates(x:str):
    """
    x: Format 'YYYY Q1'
    return: 'YYYY-mm-01'
    """
    year, qtr = x.rstrip().lstrip().split(' ')
    month = 1 if qtr=='Q1' else 4 if qtr=='Q2' else 7 if qtr=='Q3' else 10
    return datetime(int(year),month,1).strftime("%Y-%m-%d")

In [55]:
src_table = pd.read_excel("DataSources.xlsx",index_col=0)
sources = list(set(src_table.SRC.to_list()))

In [56]:
sources

['usdeur_actual',
 'gdp_forecast',
 'fert_forecast',
 'usdeur_forecast',
 'fpi_actual',
 'ng_actual',
 'ng_forecast',
 'fpi_forecast',
 'fert_actual',
 'gdp_actual']

In [57]:
src_table

,API,SRC,TYPE,Download type,STATUS,S3 Location,Frequency of Run
#,,,,,,,
7.,FAO,fert_actual,A,Total Fertilizer Production,Complete,s3://insights-shared-prod-data-lake-non-hr/CLE...,Annual
9.,FAO,fert_forecast,F,Total Fertilizer Production - Forecast,Complete,s3://insights-shared-prod-data-lake-non-hr/CLE...,Annual
8.,FAO,fpi_actual,A,Food Price Index,Complete,s3://insights-shared-prod-data-lake-non-hr/CLE...,Monthly
10.,FAO,fpi_forecast,F,Food Price Index - Forecast,Complete,s3://insights-shared-prod-data-lake-non-hr/CLE...,Annual
2.,FRED,gdp_actual,A,US GDP - Historical,Complete,s3://insights-shared-prod-data-lake-non-hr/CLE...,Quarterly
5.,EIA,gdp_forecast,F,US GDP - Forecast STEO,Complete,s3://insights-shared-prod-data-lake-non-hr/CLE...,Monthly
3.,EIA,ng_actual,A,Henry Hub Gas Price Historical,Complete,s3://insights-shared-prod-data-lake-non-hr/CLE...,Monthly
4.,EIA,ng_forecast,F,Henry Hub Gas Price - Forecast STEO,Complete,s3://insights-shared-prod-data-lake-non-hr/CLE...,Monthly
1.,FRED,usdeur_actual,A,USD/EURO exchange rate - Historical,Complete,s3://insights-shared-prod-data-lake-non-hr/CLE...,Monthly


In [58]:
src_table[src_table.SRC=='gdp_actual']['S3 Location'].iloc[0]

's3://insights-shared-prod-data-lake-non-hr/CLEANED/EXTERNAL/system=fred/usecase=US_GDP/US_GDP.parquet'

#### ITERATE THROUGH EACH SOURCE

In [11]:
source = sources[0]
source

'GDP'

### GDP

In [50]:
source='GDP'
a_src = src_table[src_table.SRC==source]

actual_s3_link = a_src[a_src.TYPE=='A']['S3 Location'].iloc[0]
forecast_s3_link = a_src[a_src.TYPE=='F']['S3 Location'].iloc[0]


actual = pd.read_parquet(actual_s3_link)

forecast = pd.read_parquet(forecast_s3_link)
forecast['date'] = forecast['date'].apply(quarter_to_month)

# SAME COLUMN NAME AS THE ACTUAL SERIES
forecast['US_GDP'] = forecast.nominalGDP

# DROP USELESS COLUMNS
forecast.drop(['realGDP', 'deflator', 'nominalGDP'],axis=1,inplace=True)

# COMBINE BOTH DATAFRAMES
gdp_consolidated = pd.concat([actual,forecast],ignore_index=True)

# REMOVING DUPLICATES AND KEEPING THE ACTUAL FOR OLD FORECAST SERIES
gdp_consolidated = gdp_consolidated[~gdp_consolidated['date'].duplicated(keep='first')]

In [52]:
gdp_consolidated

,date,US_GDP
0,2014-01-01,17144.281
1,2014-04-01,17462.703
2,2014-07-01,17743.227
3,2014-10-01,17852.540
4,2015-01-01,17991.348
5,2015-04-01,18193.707
6,2015-07-01,18306.960
7,2015-10-01,18332.079
8,2016-01-01,18425.306
9,2016-04-01,18611.617


In [17]:
forecast.date.apply(lambda x:x.rstrip())

'2014 Q1 '

In [28]:
quarter_to_month('2014 Q4 ')

'2014-10-01'

In [ ]:
# READING PARQUET FILES


In [ ]:
src_table[src_table.SRC==sources[0]]


In [ ]:
# LATEST VALUE
src_actuals = src_table[src_table.TYPE=='A'] 
# raw_data = src_table[src_table.TYPE=='A'].apply(lambda x:[x.SRC,pd.read_csv(x['S3 Location'])],axis=1)

In [ ]:
src_actuals

In [ ]:
x = src_actuals.iloc[0]

In [ ]:
x['S3 Location']

In [ ]:
df = pd.read_csv(x['S3 Location'])

In [ ]:
df.dropna(inplace=True)

In [ ]:
pd.read_csv('s3://insights-shared-non-prod-data-lake/CLEANED/EXTERNAL/system=fao/usecase=totalFertilizationProductionHist/')

# EURUSD

In [ ]:
url = 's3://insights-canpotex-downloads-sandboxlz/Fred-downloads/EXUSEU/EXUSEU.csv'
df = pd.read_csv(url,index_col=0,parse_dates=True)
df # REQUIRES UPDATE!!!!!

# EURUSD - Forecast

In [ ]:
url = 's3://insights-canpotex-downloads-sandboxlz/OECD-downloads/EXCHUD/EXCHUD.csv'
df = pd.read_csv(url,index_col=0,parse_dates=True)
df # Time update and resampling required 

#  US GDP

In [ ]:
url = 's3://insights-canpotex-downloads-sandboxlz/Fred-downloads/US_GDP/US_GDP.csv'
df = pd.read_csv(url,index_col=0,parse_dates=True)
df #Requires update | Resample to monthly | Index as datetime

In [ ]:
actual | forecast | consolidated

# US GDP - forecast

In [ ]:
url = 's3://insights-canpotex-downloads-sandboxlz/EIA-downloads/HenryHubGasPrice/nominalGDP.csv'
df = pd.read_csv(url,index_col=0,parse_dates=True)
df # Resample to monthly | Index as Datetime | Why do we have Qtr pricing (historical)

# NATURAL GAS PRICES

In [ ]:
url = 's3://insights-canpotex-downloads-sandboxlz/EIA-downloads/HenryHubGasPrice/gasprice.csv'
df = pd.read_csv(url,index_col=0,parse_dates=True)
df #Requires update | Resample to monthly | Index as datetime

# NATURAL GAS FORECAST

In [ ]:
url = 's3://insights-canpotex-downloads-sandboxlz/EIA-downloads/HenryHubGasPrice/gaspricesteo.csv'
df = pd.read_csv(url,index_col=0,parse_dates=True)
df # (need a tag for last updated) | Why do we have quarterly pricing above?

# TOTAL FERTILIZER PRODUCTION

In [ ]:
url = 's3://insights-canpotex-downloads-sandboxlz/FAO-downloads/totalFertilizationProductionHist/totalFertilizerProduction.csv'
df = pd.read_csv(url,index_col=0,parse_dates=True)
df # No dates...

# TOTAL FERTILIZER PRODUCTION FORECAST

In [ ]:
url = None
df = pd.read_csv(url,index_col=0,parse_dates=True)
df # (need a tag for last updated) | Why do we have quarterly pricing above?

# FOOD PRICE INDEX

In [ ]:
url = 's3://insights-canpotex-downloads-sandboxlz/FAO-downloads/FoodPriceIndex/FoodPriceIndex-hist.csv'
df = pd.read_csv(url,index_col=0,parse_dates=True)
df #Requires cleanup NaN values and index as datetime

# FOOD PRIE INDEX FORECAST

In [ ]:
url = 's3://insights-canpotex-downloads-sandboxlz/FAO-downloads/FoodPriceIndex/FoodPriceIndex-forecast.csv'
df = pd.read_csv(url,index_col=0,parse_dates=True)
df #Requires cleanup NaN values and index as datetime